<a href="https://colab.research.google.com/github/RV-delta/YOLOv4-Cloud-Tutorial/blob/master/dirt_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

Files needed in the yolov4 folder:

Labeled custom dataset:
- Train data (obj.zip)
- Validation data (valid.zip)
- Test data (test.zip)

Configutation file (yolov4-obj.cfg)

obj.data and obj.names files

train.txt file



In [ ]:
#check what is in the file
!ls /content/drive/MyDrive/yolov4


In [ ]:
#copy over both datasets into the root directory of the Colab VM
#change test to valid folder in drive
!cp /content/drive/MyDrive/yolov4/obj.zip ../
!cp /content/drive/MyDrive/yolov4/test.zip ../
!cp /content/drive/MyDrive/yolov4/test2.zip ../


In [ ]:
#unzip datasets and it will be stored in the /darknet/data folder
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/
!unzip ../test2.zip -d data/

In [ ]:
#!!only in case you start a new cfg file!!
# download cfg to google drive and change its name
#!cp cfg/yolov4-custom.cfg /content/drive/MyDrive/yolov4/yolov4-obj.cfg

Now you need to edit the .cfg to fit your needs based on your object detector. Open it up in a code or text editor to do so.

The necessary parameters are explained in the thesis.


change the yolov4-obj file

In [ ]:
# upload the custom .cfg back to cloud VM from Google Drive
!cp /content/drive/MyDrive/yolov4/yolov4-obj.cfg ./cfg
#!cp /content/drive/MyDrive/yolov4/Testing_With_All_Classes/full_data_default/yolov4-obj.cfg ./cfg

### obj.names and obj.data 


In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive

!cp /content/drive/MyDrive/yolov4/obj.names ./data
!cp /content/drive/MyDrive/yolov4/obj.data ./data


## iii) Generating train.txt and test.txt

In [ ]:
# upload the generate_train.py and generate_test.py script to cloud VM from Google Drive
!cp /content/drive/MyDrive/yolov4/generate_train.py ./
!cp /content/drive/MyDrive/yolov4/generate_test.py ./

In [ ]:
#run both scripts to generate the two txt files
!python generate_train.py
!python generate_test.py

In [ ]:
# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

# Step 4: Download pre-trained weights for the convolutional layers.
This step downloads the weights for the convolutional layers of the YOLOv4 network. By using these weights it helps your custom object detector to be way more accurate and not have to train as long. You don't have to use these weights but trust me it will help your modle converge and be accurate way faster. USE IT!

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137


#Train Your Custom Object Detector!

The next command will start the training of the detection model

In [ ]:
# train your custom detector

!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137  -dont_show -map

In [ ]:
# show chart.png of how custom object detector did with training
imShow('chart.png')

In [ ]:
# kick off training from where it last saved
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/yolov4/backup/yolov4-obj_best.weights  -dont_show -map

In [ ]:
#To check the results of the training based on the validation dataset
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/yolov4/Testing_With_All_Classes/full_data_default/yolov4-obj_best.weights -dont_show -image

# Step 7: Run Your Custom Object Detector


In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)

!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/yolov4/Testing_With_All_Classes/full_data_default/yolov4-obj_best.weights /content/darknet/data/test/carbon_86.jpg
imShow('predictions.jpg')